In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Author : lavinigam@, parasharsh@

In [1]:
!pip install google-cloud-aiplatform backoff --upgrade --quiet --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
import vertexai

PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

# Functions

In [3]:
from vertexai.preview.language_models import CodeChatModel
from vertexai.preview.language_models import CodeGenerationModel

In [4]:
code_generation_model = CodeGenerationModel.from_pretrained("code-bison@001")
code_completion_model = CodeGenerationModel.from_pretrained("code-gecko@001")
code_chat_model = CodeChatModel.from_pretrained("codechat-bison@001")

In [5]:
import backoff
from google.api_core.exceptions import ResourceExhausted

@backoff.on_exception(backoff.expo, ResourceExhausted, max_time=10)
def get_code_generation(code_generation_model,
                        prompt,
                        temperature: float = 0.5,
                        max_output_tokens: int = 256
) -> object:
    """Example of using Code Generation to write a function."""
    parameters = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
    }
    response = code_generation_model.predict(prefix=prompt,
                                             temperature=temperature,
                                             max_output_tokens=max_output_tokens)
    return response.text

@backoff.on_exception(backoff.expo, ResourceExhausted, max_time=10)
def get_code_completion(code_completion_model,
                        prompt,
                        temperature: float = 0.2,
                        max_output_tokens: int = 64
) -> object:
    """Example of using Code Generation to write a function."""
    parameters = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
    }
    response = code_completion_model.predict(prefix=prompt,
                                             temperature=temperature,
                                             max_output_tokens=max_output_tokens)
    return response.text

@backoff.on_exception(backoff.expo, ResourceExhausted, max_time=10)
def get_code_chat(code_chat_model,
                        prompt,
                        temperature: float = 0.2,
                        max_output_tokens: int = 1024
) -> object:
    """Example of using Code Generation to write a function."""
    parameters = {
        "temperature": temperature,
        "max_output_tokens": max_output_tokens,
    }
    chat = code_chat_model.start_chat() #initiate the session
    response = chat.send_message(prompt,
                                             temperature=temperature,
                                             max_output_tokens=max_output_tokens)
    return response.text

In [6]:
print(get_code_generation(code_generation_model,
                          prompt = "write a python function for binary tree")) #Generation

```python
def binary_tree(root):
  """
  This function creates a binary tree.

  Args:
    root: The root node of the tree.

  Returns:
    The binary tree.
  """

  # Create an empty list to store the nodes of the tree.
  nodes = []

  # Add the root node to the list.
  nodes.append(root)

  # While there are still nodes in the list,
  # continue to add their children to the list.
  while nodes:
    # Get the next node from the list.
    node = nodes.pop(0)

    # If the node has a left child,
    if node.left:
      # Add the left child to the list.
      nodes.append(node.left)

    # If the node has a right child,
    if node.right:
      # Add the right child to the list.
      nodes.append(node.right)

  # Return the binary


In [7]:
print(get_code_completion(code_completion_model,
                          prompt = "def binary_tree(root):")) #Completition


    if root is None:
        return
    stack = [root]
    while stack:


In [8]:
print(get_code_chat(code_chat_model,
                        prompt = "what is the most scalable way to traverse a list in python?")) #chat

The most scalable way to traverse a list in Python is to use the `for` loop. The `for` loop iterates over the elements of a list, and it is the most efficient way to traverse a list.

To use the `for` loop, you first need to declare a variable that will store the current element of the list. Then, you need to use the `in` keyword to check if the current element is in the list. If the current element is in the list, the `for` loop will execute the code inside the loop body.

The following is an example of how to use the `for` loop to traverse a list:

```
# Declare a variable to store the current element of the list.
current_element = None

# Check if the current element is in the list.
if current_element in the_list:
    # Execute the code inside the loop body.
    pass
```

The `for` loop is the most scalable way to traverse a list because it is efficient and it does not require any additional memory.


# Code Generation

## Python - Data Science

In [9]:
prompt = f"""
         write a python script that reads the csv file "mydata.csv" using pandas library read_csv function. \n
         then change the column names to ["col_1", "col_2", "col_3"].
         """
print(get_code_generation(code_generation_model,prompt = prompt))

```python
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("mydata.csv")

# Change the column names
df.columns = ["col_1", "col_2", "col_3"]

# Print the DataFrame
print(df)
```


In [10]:
prompt = f"""write a python code to plot a simple histrogram using seaborn library and histplot function that takes the dataframe "user_df".
              The "user_df" has "col_1" as ["A", "B", "C", "D", "E"] and "col_2" has flaot values.
              Add atleast 50 examples in the sample "user_df".
              Import all necessary libraries. create a histogram by taking x axis as "col_1" and no y axis.
          """
print(get_code_generation(code_generation_model,prompt = prompt))

```python
import seaborn as sns
import pandas as pd

# Create a sample dataframe
user_df = pd.DataFrame({
    "col_1": ["A", "B", "C", "D", "E"],
    "col_2": [1.0, 2.0, 3.0, 4.0, 5.0],
})

# Plot the histogram
sns.histplot(data=user_df, x="col_1", y="col_2")
```


## Python - General

In [11]:
prompt = f"""write well tested python function where given a string containing just the characters '(' and ')',
            return the length of the longest valid parentheses substring """
print(get_code_generation(code_generation_model,prompt = prompt))

```python
def longest_valid_parentheses(string):
    """
    This function returns the length of the longest valid parentheses substring in a given string.

    Args:
        string: A string containing only the characters '(' and ')'.

    Returns:
        The length of the longest valid parentheses substring in the given string.
    """

    # Initialize the stack and the current longest substring length.
    stack = []
    max_length = 0

    # Iterate over the characters in the string.
    for character in string:
        # If the character is '(', push it onto the stack.
        if character == '(':
            stack.append(character)

        # If the character is ')', pop an element from the stack.
        else:
            if stack:
                stack.pop()

                # If the stack is empty, the current substring is not valid.
                if not stack:
                    max


In [12]:
prompt = f"""write a python function named as "calculate_cosine_similairty" where it takes two arguments "vector1" and "vector2".
             It then uses numpy dot function to calculate the dot product of the two vectors. \n
             Also write three unit test for the function.
            """
print(get_code_generation(code_generation_model,prompt = prompt))

```python
def calculate_cosine_similarity(vector1, vector2):

    """
    This function calculates the cosine similarity between two vectors.

    Args:
        vector1 (numpy.ndarray): The first vector.
        vector2 (numpy.ndarray): The second vector.

    Returns:
        float: The cosine similarity between the two vectors.
    """

    # Calculate the dot product of the two vectors.
    dot_product = np.dot(vector1, vector2)

    # Calculate the magnitude of the two vectors.
    vector1_magnitude = np.linalg.norm(vector1)
    vector2_magnitude = np.linalg.norm(vector2)

    # Calculate the cosine similarity.
    cosine_similarity = dot_product / (vector1_magnitude * vector2_magnitude)

    return cosine_similarity

```


## Rust

In [13]:
prompt = f"""write a Rust function named as "calculate_cosine_similairty" where it takes two arguments "vector1" and "vector2".
              It then calculates the dot product of the two vectors and returns the value. \n
              Write steps to call the function for the user.
          """
print(get_code_generation(code_generation_model,prompt = prompt))

```rust
fn calculate_cosine_similarity(vector1: Vec<f64>, vector2: Vec<f64>) -> f64 {
    let dot_product = vector1.iter().zip(vector2.iter()).map(|(a, b)| a * b).sum();
    let magnitude1 = vector1.iter().map(|a| a.powi(2)).sum().sqrt();
    let magnitude2 = vector2.iter().map(|a| a.powi(2)).sum().sqrt();
    dot_product / (magnitude1 * magnitude2)
}
```


## Docker

In [14]:
prompt = f"""write a docker script where there is a simple linux machine that has python 3.8 installed with following libraries: pandas, tensorflow=2.11, numpy"""
print(get_code_generation(code_generation_model,prompt = prompt))

```
FROM ubuntu:20.04

RUN apt-get update && apt-get install -y python3.8 python3-pip

RUN pip3 install pandas tensorflow==2.11 numpy

CMD ["python3"]
```


## SQL

In [15]:
prompt = f"""write a standard SQL function that strips all non-alphabet characters from the string and encodes it to utf-8"""
print(get_code_generation(code_generation_model,prompt = prompt))

```sql
CREATE FUNCTION dbo.StripNonAlphaAndEncodeToUTF8(
  @InputString VARCHAR(MAX)
)
RETURNS VARCHAR(MAX)
AS
BEGIN
  -- Strip all non-alphabet characters from the string.
  SET @InputString = REPLACE(@InputString, '[^a-zA-Z]', '');

  -- Encode the string to UTF-8.
  SET @InputString = CONVERT(VARCHAR(MAX), @InputString, 1);

  -- Return the encoded string.
  RETURN @InputString;
END;
```


In [16]:
prompt = """
Generate a standard SQL function to solve given problem:

A U.S graduate school has students from Asia, Europe and America. The students' location information are stored in table student as below.

| name   | continent |
|--------|-----------|
| Jack   | America   |
| Pascal | Europe    |
| Xi     | Asia      |
| Jane   | America   |

Pivot the continent column in this table so that each name is sorted alphabetically and displayed underneath its corresponding continent.
The output headers should be America, Asia and Europe respectively. It is guaranteed that the student number from America is no less than either Asia or Europe.

"""
print(get_code_generation(code_generation_model,prompt = prompt))

```sql
CREATE FUNCTION pivot_continent(
    student_table)
RETURNS TABLE (
    continent VARCHAR(255),
    name VARCHAR(255)
) AS
BEGIN
SELECT
    continent,
    name
FROM
    student_table
ORDER BY
    continent,
    name
END;
```


## Javascript, HTML, CSS

In [17]:
prompt = """write a simple chat framework template using javascript, css and html.
The basic functionality should include an interface where user inputs can be taken and it
should respond with model generated outputs after making a curl call to the PaLM model.
"""
## code generator gives only HTML
# print(get_code_generation(code_generation_model,prompt = prompt,
#                           temperature=0.9, max_output_tokens=1024))

print(get_code_chat(code_chat_model,prompt,
                          temperature=0.9, max_output_tokens=1024))

Sure, here is a simple chat framework template using javascript, css, and html that includes an interface where user inputs can be taken and it responds with model generated outputs after making a curl call to the PaLM model:

```
<!DOCTYPE html>
<html>
<head>
  <title>Chat Framework</title>
  <link rel="stylesheet" href="style.css">
</head>
<body>
  <h1>Chat with PaLM</h1>
  <form action="/chat" method="post">
    <input type="text" name="message" placeholder="Enter your message here...">
    <input type="submit" value="Send">
  </form>

  <script src="chat.js"></script>
</body>
</html>
```

The `chat.js` file contains the following code:

```
// This is the chat framework.

// The `onmessage` event handler is called whenever a message is received from the server.
// In this case, the message will be the user's input.
window.addEventListener('message', function(event) {
  var message = event.data;

  // The `chat` function is used to send a message to the server.
  chat(message);
});


In [ ]:
prompt = """write a simple web page using javascript, html and css where user can upload any image file.
Once the file is uploaded, it should be saved in GCS bucket. After that it should call "get_caption_model" API and
returns the caption of the input image which is then displayed to the user along with the image on screen. The display should be
at the center.
"""
print(get_code_chat(code_chat_model,prompt,
                          temperature=0.9, max_output_tokens=1024))

Here is a simple web page that you can use to upload an image file, save it in a GCS bucket, and then call the "get_caption_model" API to get the caption for the image. The caption will be displayed to the user along with the image on the screen.

The code for the web page is below:

```html
<!DOCTYPE html>
<html>
<head>
  <title>Image Captioning</title>
</head>
<body>
  <h1>Image Captioning</h1>

  <p>This web page allows you to upload an image file, save it in a GCS bucket, and then call the "get_caption_model" API to get the caption for the image. The caption will be displayed to the user along with the image on the screen.</p>

  <form action="/upload" method="post" enctype="multipart/form-data">
    <input type="file" name="file">
    <input type="submit" value="Upload">
  </form>

  <div id="image"></div>

  <div id="caption"></div>

</body>
</html>
```

When the user uploads an image file, the file will be saved in the GCS bucket with the following path:

```gs://bucket_name/ima

## C++

In [ ]:
language = "C++ function"
file_format = "json"
extract_info = "names"
requirments = """
- the name should be start with capital letters.
- There should be no duplicate names in the final list.
"""

prompt = f"""Create a {language} to parse {file_format} and extract {extract_info} with the following requirements: {requirments}.
              """

print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))


```c++
#include <iostream>
#include <string>
#include <vector>

using namespace std;

// This function parses a JSON object and extracts all of the names.
// The names are returned in a vector of strings.
vector<string> getNames(const string& json) {
  // Create a vector to store the names.
  vector<string> names;

  // Create a JSON parser.
  json_parser parser;

  // Parse the JSON object.
  json_value value = parser.parse(json);

  // Iterate over the object's properties.
  for (json_object::iterator it = value.begin(); it != value.end(); ++it) {
    // Get the property name.
    string name = it->first;

    // Check if the name is already in the vector.
    if (find(names.begin(), names.end(), name) == names.end()) {
      // The name is not in the vector, so add it.
      names.push_back(name);
    }
  }

  // Return the vector of names.
  return names;
}
```


## C'#'

In [ ]:
language = "C# function"
file_format = "txt"
extract_info = "address"
requirments = """
- the address should include pin codes.
- There should be no duplicate addresses in the final list.
"""

prompt = f"""Create a {language}  to parse {file_format} and extract {extract_info} with the following requirements: {requirments}.
              """
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

```c#
public static List<string> GetAddressesWithPinCodes(string filePath)
{
    // Create a list to store the addresses.
    List<string> addresses = new List<string>();

    // Open the txt file.
    StreamReader reader = new StreamReader(filePath);

    // Read each line of the file.
    string line;
    while ((line = reader.ReadLine()) != null)
    {
        // Split the line into words.
        string[] words = line.Split(" ");

        // Check if the line contains a pin code.
        if (words.Contains("pin code"))
        {
            // Add the address to the list.
            addresses.Add(line);
        }
    }

    // Close the file.
    reader.Close();

    // Remove any duplicate addresses from the list.
    HashSet<string> uniqueAddresses = new HashSet<string>(addresses);
    addresses = new List<string>(uniqueAddresses);

    // Return the list of addresses.
    return addresses;
}
```


## Java

In [ ]:
language = "java function"
file_format = "txt"
extract_info = "address"
requirments = """
- the address should include pin codes.
- There should be no duplicate addresses in the final list.
"""

prompt = f"""Create a {language} to parse {file_format} and extract {extract_info} with the following requirements: {requirments}.
              """
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

```java
import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.util.ArrayList;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

public class AddressParser {

    public static List<String> parseAddresses(String filePath) throws IOException {
        // Create a buffered reader to read the text file.
        BufferedReader reader = new BufferedReader(new FileReader(filePath));

        // Create a list to store the addresses.
        List<String> addresses = new ArrayList<>();

        // Create a map to store the addresses and their pin codes.
        Map<String, String> addressPinCodeMap = new HashMap<>();

        // Read each line from the text file.
        String line;
        while ((line = reader.readLine()) != null) {
            // Split the line into tokens.
            String[] tokens = line.split(",");

            // Get the address and the pin code.
            String address = tokens[0];
            String

## GO

In [ ]:
language = "go lang server script"
file_format = " user uploaded txt file"
extract_info = "ip addresses"
requirments = """
- the format of ip addresses should be standard ipv4
- There should be no duplicate ip addresses in the final list.
- The server should return the final list of ip addresses which are in correct format and follows IPV4 standard
"""

prompt = f"""Create a {language} to parse {file_format} and extract {extract_info} with the following requirements: {requirments}.
              """
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

```
package main

import (
    "fmt"
    "io/ioutil"
    "net"
)

func main() {
    // Get the file contents from the user.
    file, err := ioutil.ReadFile("input.txt")
    if err != nil {
        panic(err)
    }

    // Split the file contents into lines.
    lines := strings.Split(string(file), "\n")

    // Create a slice to store the IP addresses.
    ips := make([]string, 0)

    // Iterate over the lines and extract the IP addresses.
    for _, line := range lines {
        // Check if the line is a valid IP address.
        if net.ParseIP(line) == nil {
            continue
        }

        // Check if the IP address is already in the list.
        if !contains(ips, line) {
            // The IP address is not in the list, so add it.
            ips = append(ips, line)
        }
    }

    // Sort the IP addresses.
    sort.Strings(ips)

    // Print the IP addresses to the console.
    for _, ip := range ips {
        fmt.Println(ip)
    }
}

// contains checks if a given s

## Kotlin

In [ ]:
language = "kotlin script"
file_format = " user uploaded image through android camera capture"
extract_info = "face emotion through GoogleImageEmptionClassification endpoint"
requirments = """
- the image should be in standrd format as expected by GoogleImageEmptionClassification /checkimage endpoint.
- the respons of the model should overlay the image uploade dby the user
"""

prompt = f"""Create a {language} to parse {file_format} and extract {extract_info} with the following requirements: {requirments}.
              """
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

```kotlin
import android.Manifest
import android.content.pm.PackageManager
import android.graphics.Bitmap
import android.graphics.BitmapFactory
import android.os.Bundle
import android.util.Log
import android.widget.ImageView
import androidx.appcompat.app.AppCompatActivity
import androidx.core.app.ActivityCompat
import com.google.android.gms.tasks.Task
import com.google.firebase.ml.vision.FirebaseVision
import com.google.firebase.ml.vision.common.FirebaseVisionImage
import com.google.firebase.ml.vision.face.FirebaseVisionFace
import com.google.firebase.ml.vision.face.FirebaseVisionFaceDetector
import com.google.firebase.ml.vision.face.FirebaseVisionFaceLandmark

class MainActivity : AppCompatActivity() {

    private lateinit var imageView: ImageView

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        imageView = findViewById(R.id.imageView)

        // Check if the app has the requi

## BigQuery

In [ ]:
prompt = """
Your are an expert developer in Google BigQuery SQL and BigQuery machine Learning.
Write the SQL code to get list of table names in BigQuery dataset using below information.
Only use columns that are present in the table. Do not use any other columns.
For details on information Schema, search this URL https://cloud.google.com/bigquery/docs/information-schema-intro.
dataset: retail
project_id: rvd-gcp
"""
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))


```sql
SELECT table_name
FROM `rvd-gcp.retail.INFORMATION_SCHEMA.TABLES`
WHERE dataset_name = 'retail'
```


In [ ]:
prompt = """
You are an expert in GoogleSQL and BigQuery SQL. Generate a syntactically correct SQL Query to find column level schema details using the information given below.

Dataset Details:
dataset: retail
project_id: rvd-gcp
table_names: customers, order_items, products

Only use the columns mentioned below in the output SQL Query.

Output Column names:
column_name, data_type, table_name
"""
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

```sql
SELECT
  column_name,
  data_type,
  table_name
FROM
  `rvd-gcp.retail.customers`
UNION ALL
SELECT
  column_name,
  data_type,
  table_name
FROM
  `rvd-gcp.retail.order_items`
UNION ALL
SELECT
  column_name,
  data_type,
  table_name
FROM
  `rvd-gcp.retail.products`;
```


In [ ]:
prompt = """
You are an expert in GoogleSQL and BigQuery SQL. As an analyst you are asked to prepare reports to summarize sales performance across products, customers, demographics, order value.
Generate syntactically SQL Query to create an summary view using multiple columns from different tables. Use fully qualified bigquery table names along with gcp project_id, dataset name.
Only use the column names present in the above listed table schema. Do not use any other columns.

Dataset Details:
dataset: retail
project_id: rvd-gcp

Table Details:
['column_name', 'data_type', 'table_name']
['address', 'STRING', 'customers']
['age', 'INT64', 'customers']
['city', 'STRING', 'customers']
['country', 'STRING', 'customers']
['created_at', 'TIMESTAMP', 'customers']
['email', 'STRING', 'customers']
['first_name', 'STRING', 'customers']
['gender', 'STRING', 'customers']
['id', 'INT64', 'customers']
['last_name', 'STRING', 'customers']
['latitude', 'FLOAT64', 'customers']
['longitude', 'FLOAT64', 'customers']
['state', 'STRING', 'customers']
['traffic_source', 'STRING', 'customers']
['zip', 'STRING', 'customers']
['CREATED_AT', 'TIMESTAMP', 'order_items']
['DELIVERED_AT', 'TIMESTAMP', 'order_items']
['ID', 'INT64', 'order_items']
['INVENTORY_ITEM_ID', 'INT64', 'order_items']
['ORDER_ID', 'INT64', 'order_items']
['RETURNED_AT', 'TIMESTAMP', 'order_items']
['SALE_PRICE', 'FLOAT64', 'order_items']
['SHIPPED_AT', 'TIMESTAMP', 'order_items']
['STATUS', 'STRING', 'order_items']
['USER_ID', 'INT64', 'order_items']
['BRAND', 'STRING', 'products']
['CATEGORY', 'STRING', 'products']
['COST', 'FLOAT64', 'products']
['DEPARTMENT', 'STRING', 'products']
['DISTRIBUTION_CENTER_ID', 'INT64', 'products']
['ID', 'INT64', 'products']
['NAME', 'STRING', 'products']
['RETAIL_PRICE', 'FLOAT64', 'products']
['SKU', 'STRING', 'products']
"""

print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))


```sql
SELECT
  t1.*,
  t2.*,
  t3.*
FROM
  `rvd-gcp.retail.customers` AS t1
  JOIN
  `rvd-gcp.retail.order_items` AS t2
    ON t1.id = t2.user_id
  JOIN
  `rvd-gcp.retail.products` AS t3
    ON t2.inventory_item_id = t3.id
```


# Code Completion

## Python

In [ ]:
prompt = f"""def find_x_in_string(string_s, x):"""

print(get_code_completion(code_completion_model,
                          prompt = prompt))


    """
    Find the index of the x in a string.
    """
    for i in range(len(string_s)):
        if string_s[i] == x:
            return i
    return -1


def find_y_in_string(


# Code error/bug fix

## Python

In [ ]:
prompt = f"""explain the error in the below code: \n\n
mystr = "PaLM"
num = 4
print mystr + num"""

print(get_code_chat(code_chat_model,prompt,
                          temperature=0.5, max_output_tokens=1024))

The error is that you cannot concatenate a string and an integer. You can convert the integer to a string, or you can use string formatting.

To convert the integer to a string, you can use the `str()` function. For example:

```
mystr = "PaLM"
num = 4
print(mystr + str(num))
```

This will print the string "PaLM4".

To use string formatting, you can use the `format()` function. For example:

```
mystr = "PaLM"
num = 4
print('{}{}'.format(mystr, num))
```

This will also print the string "PaLM4".


In [ ]:
language = "python"
code_snippet = """
                d = {2:3, 1:89, 4:5, 3:0}
                od = collections.OrderedDict(sorted(d.items()))
               """
prompt = f"Identify potential performance issues in the given {language} code: {code_snippet}."
print(get_code_chat(code_chat_model,prompt,
                          temperature=0.5, max_output_tokens=1024))

The code you provided sorts the dictionary `d` in ascending order based on the keys. This can be a potential performance issue if the dictionary is large. A more efficient way to sort a dictionary is to use the `heapq` module. The `heapq` module provides a number of functions for sorting and heapifying data structures. In this case, you can use the `heapify` function to create a heap from the keys of the dictionary, and then use the `heappop` function to repeatedly remove the smallest key from the heap. This will give you the keys in sorted order, without having to sort the entire dictionary.

Here is an example of how to use the `heapq` module to sort a dictionary:

```python
import heapq

d = {2: 3, 1: 89, 4: 5, 3: 0}

# Create a heap from the keys of the dictionary.
heap = [(v, k) for k, v in d.items()]
heapq.heapify(heap)

# Repeatedly remove the smallest key from the heap.
while heap:
    key, value = heapq.heappop(heap)
    print(key, value)
```

This will print the keys of the d

## C++

In [ ]:
language = "c++"
code_snippet = """
                map<string, string>::iterator itr;
                for(itr = mymap.begin(); itr != mymap.end(); itr++)
                {
                  string newline = itr->second;
                  cout << newline << endl;
                }
               """
prompt = f"Identify potential performance issues in the given {language} code: {code_snippet}."
print(get_code_chat(code_chat_model,prompt,
                          temperature=0.5, max_output_tokens=1024))

The code you have provided does not contain any obvious performance issues. However, there are a few things you could do to improve its performance.

First, you could use the `std::get` function to get the value associated with a key in the map. This would eliminate the need to create a temporary variable to store the value.

Second, you could use the `std::for_each` algorithm to iterate over the map. This would eliminate the need to declare and initialize an iterator variable.

Finally, you could use the `std::cout << std::endl` manipulator to print a newline character to the output stream. This would eliminate the need to create a temporary string variable to store the newline character.

Here is the code after these changes have been made:

```c++
#include <map>
#include <iostream>

int main() {
  // Create a map of strings.
  std::map<string, string> mymap;

  // Add some key-value pairs to the map.
  mymap["key1"] = "value1";
  mymap["key2"] = "value2";
  mymap["key3"] = "value3";

# Code Optimization

## Python

In [ ]:
prompt = f"""can you write a class for the given function and also make sure that the it has strong python type hints and exception handling.\n\n\n
def deploy_model(model_id, task):
    model_name = "blip-image-captioning"
    endpoint = aiplatform.Endpoint.create(display_name=f"{{model_name}}-endpoint")
    serving_env = {{
        "MODEL_ID":,
        "TASK":,
    }}
    # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
    artifact_uri = model_id if model_id.startswith("gs://") else None
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/transformers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
        artifact_uri=artifact_uri,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-8",
        accelerator_type="NVIDIA_TESLA_T4",
        accelerator_count=1,
        deploy_request_timeout=1800,
    )
    return model, endpoint
  """
print(get_code_chat(code_chat_model,prompt,
                          temperature=0.8, max_output_tokens=1024))

Sure, here is the class for the given function:

```python
class DeployModel:

    def __init__(self, model_id: str, task: str):
        """
        Initialize the DeployModel class.

        Args:
            model_id: The ID of the model to deploy.
            task: The task of the model to deploy.
        """
        self.model_id = model_id
        self.task = task

    def deploy(self):
        """
        Deploy the model to AI Platform.

        Returns:
            The model and endpoint objects.
        """
        model_name = "blip-image-captioning"
        endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
        serving_env = {
            "MODEL_ID": self.model_id,
            "TASK": self.task,
        }
        # If the model_id is a GCS path, use artifact_uri to pass it to serving docker.
        artifact_uri = self.model_id if self.model_id.startswith("gs://") else None
        model = aiplatform.Model.upload(
            display_name=model_

# Code Explanation

## Python

In [ ]:
prompt = """can you explain this python function line by line:

def linear_Search(list1, n, key):
    for i in range(0, n):
        if (list1[i] == key):
            return i
    return -1
"""
# print(get_code_chat(prompt=prompt,temperature=0.5,maxOutputTokens=1024))
print(get_code_chat(code_chat_model,prompt,
                          temperature=0.5, max_output_tokens=1024))

The `linear_Search` function takes in a list, `list1`, an integer, `n`, which is the length of the list, and a key, `key`. The function then iterates through the list, starting at index 0, and comparing each element to the key. If the element at index `i` is equal to the key, the function returns `i`. If the function reaches the end of the list without finding the key, it returns -1.

Here is a more detailed explanation of the steps involved in the `linear_Search` function:

1. The function first iterates through the list, starting at index 0.
2. At each iteration, the function compares the element at index `i` to the key.
3. If the element at index `i` is equal to the key, the function returns `i`.
4. If the function reaches the end of the list without finding the key, it returns -1.


# Code Translation

## Python-Rust

In [ ]:
prompt = f"""can you convert the following python code that removes all numbers from the string to rust function:

def remove_digits(s):
      result = ''.join([i for i in s if not i.isdigit()])
      return result
    """
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

```rust
fn remove_digits(s: &str) -> String {
    let mut result = String::new();
    for c in s.chars() {
        if !c.is_digit() {
            result.push(c);
        }
    }
    return result;
}
```


## C++ - Java

In [ ]:
#add extra "{" and "}" around original "{" "}"
prompt = f"""can you convert the following c++ code that removes all punctuations from string to java:

#include <string>
#include <iostream>
#include <cctype>

int main() {{

    std::string text = "this. is my string. it's here.";

    for (int i = 0, len = text.size(); i < len; i++)
    {{
        if (ispunct(text[i]))
        {{
            text.erase(i--, 1);
            len = text.size();
        }}
    }}

    std::cout << text;
    return 0;
}}
"""
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

import java.util.Scanner;

public class RemovePunctuation {

    public static void main(String[] args) {

        Scanner input = new Scanner(System.in);

        System.out.println("Enter a string: ");
        String text = input.nextLine();

        for (int i = 0, len = text.length(); i < len; i++) {
            if (Character.isPunctuation(text.charAt(i))) {
                text.deleteCharAt(i--);
                len = text.length();
            }
        }

        System.out.println("The new string is: " + text);
    }
}


## AWS Terraform - GCP Terraform

In [ ]:
prompt = f"""Convert the following Terraform code to work in GCP:
data "aws_ami" "my_base" {{ \n
  most_recent = true \n
  owners      = ["self"] \n
}}
"""
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

```
provider "google" {
  project = var.project_id
}

data "google_compute_image" "my_base" {
  most_recent = true
  family = "debian-cloud"
}
```


# Code test cases

## Python

In [ ]:
prompt = f"""can you write five test cases python functions for the bellow python functions:
def linear_Search(list1, n, key):
    for i in range(0, n):
        if (list1[i] == key):
            return i
    return -1
    """
print(get_code_generation(code_generation_model,prompt = prompt,
                          temperature=0.5, max_output_tokens=1024))

Here are five test cases for the linear_Search() function:

1. Test case 1:

    * Input: list1 = [1, 2, 3, 4, 5], n = 5, key = 3
    * Expected output: 2

2. Test case 2:

    * Input: list1 = [1, 2, 3, 4, 5], n = 5, key = 10
    * Expected output: -1

3. Test case 3:

    * Input: list1 = ['a', 'b', 'c', 'd', 'e'], n = 5, key = 'c'
    * Expected output: 2

4. Test case 4:

    * Input: list1 = ['a', 'b', 'c', 'd', 'e'], n = 5, key = 'f'
    * Expected output: -1

5. Test case 5:

    * Input: list1 = [], n = 0, key = 'a'
    * Expected output: -1


In [ ]:
prompt = """
Generate three unit test for Python function:


size=int(input("Enter the number of elements you want in array: "))
arr=[]
#taking input of the list
for i in range(0,size):
    elem=int(input("Please give value for index "+str(i)+": "))
    arr.append(elem)
#taking average of the elements of the list
avg=sum(arr)/size
print("Average of the array elements is ",avg)

"""
print(get_code_chat(code_chat_model,prompt,
                          temperature=0.9, max_output_tokens=1024))

Here are three unit tests for the Python function:

1. Test for empty array:

```
def test_empty_array():
    assert average_array([]) == 0.0
```

2. Test for array with one element:

```
def test_array_with_one_element():
    assert average_array([1]) == 1.0
```

3. Test for array with multiple elements:

```
def test_array_with_multiple_elements():
    assert average_array([1, 2, 3]) == 2.0
```
